In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    #spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38') 

23/05/10 21:24:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0712:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230510-2124-0.2.113-cf32652c5077.log


In [3]:
from hail.ggplot import *

import plotly
import plotly.io as pio
pio.renderers.default='iframe'

In [ ]:
full = hl.vds.read_vds('/net/ascratch/people/plggosborcz/full-rpkm.vds')

vd_gt = full.variant_data.transmute_entries(
    GT = hl.vds.lgt_to_gt(full.variant_data.LGT, full.variant_data.LA)
)

mt = hl.vds.to_dense_mt(hl.vds.VariantDataset(full.reference_data, vd_gt))
mt = mt.checkpoint(localfs_path+'full-dense.mt')

mt = hl.sample_qc(mt)

mt = mt.annotate_cols(
    group = hl.if_else(
        mt.s.contains('B'),
        'local_controls',
        hl.if_else(
            (mt.s.contains('NA') | mt.s.contains('HG')),
            '1kg_controls',
            'GTS'
        )
    )
)  
    
mt = mt.filter_cols(mt.s != '494') #one sample removed due to quality issues
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

# filter by gnomad coverage
cov = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/gnomad/gnomad.genomes.r3.0.1.coverage.ht/')
cov = cov.filter(cov.over_1 > 0.9)
mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)

mt = mt.checkpoint(localfs_path+'full-dense-filtered.mt')
mt = mt.naive_coalesce(5000)
mt = mt.checkpoint(localfs_path+'full-dense-repartitioned.mt')

2023-05-10 22:43:45.960 Hail: INFO: wrote matrix table with 20108617 rows and 789 columns in 21940 partitions to /localfs/2603338/full-dense.mt


### s_vs_gts

In [ ]:
mt = hl.read_matrix_table(localfs_path+'full-dense-repartitioned.mt')

In [ ]:
# now check how many variants are here

In [16]:
#perform per_group variant QC - dp and gq stats + hwe
mt = mt.annotate_rows(dp_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.DP)),
                      gq_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)),
                      hwe = hl.agg.group_by(mt.group, hl.agg.hardy_weinberg_test(mt.GT)))

In [17]:
mt = mt.annotate_rows(n_below_dp_3 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.DP < 3)),
                      n_below_gq_30 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GQ <30)))

mt = mt.filter_rows((mt.dp_qc.get('GTS', hl.struct(mean=0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.dp_qc.get('local_controls', hl.struct(mean=0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.gq_qc.get('GTS', hl.struct(mean=0.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 40) &
                    (mt.gq_qc.get('local_controls', hl.struct(mean=0.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 40) &
                    (mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 1E-15) &
                    (mt.hwe.get('local_controls', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 1E-15) &
                    (mt.n_below_dp_3.get('local_controls', 0) < 8) &
                    (mt.n_below_gq_30.get('local_controls', 0) < 40) &
                    (mt.n_below_dp_3.get('GTS', 0) < 8) &
                    (mt.n_below_gq_30.get('GTS', 0) < 40))

In [ ]:
mt = hl.split_multi_hts(mt)
mt = mt.checkpoint(localfs_path+'mt_GTS_s_temp.mt')

In [30]:
mt = hl.read_matrix_table(localfs_path+'mt_GTS_s_temp.mt')

In [31]:
gnomad = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/gnomad/gnomad.genomes.v3.1.1.sites.ht/')
cadd = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/cadd.ht')
vep = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/vep38/grch38_context_vep_annotated.ht')

In [32]:
mt = mt.annotate_rows(gnomad_v_3_1 = gnomad[mt.row_key])
mt = mt.annotate_rows(cadd = cadd[mt.row_key])
mt = mt.annotate_rows(vep = vep[mt.row_key])

In [ ]:
mt = mt.checkpoint(localfs_path+'mt_GTS_s_temp_2.mt')

In [ ]:
mt = mt.filter_cols(
    (mt.group == 'GTS') | (mt.group == 'local_controls')
)
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [ ]:
mt.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt')

### gts only (for synthetic controls)

In [ ]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt')

In [ ]:
mt = mt.filter_cols(
    mt.group == 'GTS'
)
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [ ]:
mt.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_only.mt')

In [ ]:
# now proceed do add synthetic controls and to SKAT (these are already filtered for +/- 20kb from protein coding genes